# EQUITY AMERICAN OPTIONS

Valuation and Risk of American-style options on equity

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from financepy.utils import *
from financepy.market.curves import *
from financepy.products.equity import *
from financepy.models.black_scholes import *

####################################################################
# FINANCEPY BETA Version 0.270 - This build:  26 Feb 2023 at 19:12 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



# Define the Call and Put Options

In [ ]:
valuation_date = Date(1, 1, 2015)

In [ ]:
expiry_date = valuation_date.add_years(0.5)

In [ ]:
expiry_date

In [ ]:
strike_price = 50.0

In [ ]:
EU_CALL = OptionTypes.EUROPEAN_CALL
EU_PUT = OptionTypes.EUROPEAN_PUT

Let's create a put and a call using the Vanilla class

In [ ]:
europeanCallOption = EquityVanillaOption(expiry_date, strike_price, EU_CALL)

In [ ]:
europeanPutOption = EquityVanillaOption(expiry_date, strike_price, EU_PUT)

Now let's create a put and a call American and European option types using the American class

In [ ]:
europeanAmericanCallOption = EquityAmericanOption(expiry_date, strike_price, EU_CALL)

In [ ]:
europeanAmericanPutOption = EquityAmericanOption(expiry_date, strike_price, EU_PUT)

In [ ]:
americanCallOption = EquityAmericanOption(expiry_date, strike_price, OptionTypes.AMERICAN_CALL)

In [ ]:
americanPutOption = EquityAmericanOption(expiry_date, strike_price, OptionTypes.AMERICAN_PUT)

In [ ]:
print(americanCallOption)

In [ ]:
print(americanPutOption)

## Valuation

In [ ]:
stock_price = 50.0
volatility = 0.20
interest_rate = 0.05
dividend_yield = 0.0

In [ ]:
discount_curve = DiscountCurveFlat(valuation_date, interest_rate)

In [ ]:
dividend_curve = DiscountCurveFlat(valuation_date, dividend_yield)

#### Analytic Model

In [ ]:
model = BlackScholes(volatility)

In [ ]:
europeanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
europeanAmericanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

#### Finite Difference Model

In [ ]:
params = {
    'num_samples': 2000,
    'theta': 0.5
}
model = BlackScholes(volatility,
                     implementationType=BlackScholesTypes.FINITE_DIFFERENCE,
                     params=params)

In [ ]:
europeanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
europeanAmericanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
americanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
europeanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
europeanAmericanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
americanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

#### PSOR Model

In [ ]:
params = {
    'theta': 0.5,
    'num_samples': 2000,
}
model = BlackScholes(volatility,
                     implementationType=BlackScholesTypes.PSOR,
                     params=params)

In [ ]:
europeanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
europeanAmericanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
americanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
europeanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
europeanAmericanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
americanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

#### CRR Tree Model
These prices should be the same. If we insist on using the Tree the price changes a bit

In [ ]:
model = BlackScholes(volatility, BlackScholesTypes.CRR_TREE)

In [ ]:
europeanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
europeanAmericanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
americanCallOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
europeanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
europeanAmericanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

These prices should be the same even though one is using the BS formula and the other uses the CRR tree

In [ ]:
americanPutOption.value(valuation_date, stock_price, discount_curve, dividend_curve, model)

This price is higher as there is an advantage to exercising early when holding an American put

## Risk Measures

You can calculate the risk measures using using perturbatory analysis

In [ ]:
americanCallOption.delta(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
americanCallOption.gamma(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
americanCallOption.theta(valuation_date, stock_price, discount_curve, dividend_curve, model)

In [ ]:
americanCallOption.rho(valuation_date, stock_price, discount_curve, dividend_curve, model)

Copyright (c) 2020 Dominic O'Kane